In [1]:
import tensorflow as tf
from tensorflow.keras import Model
import numpy as np
import glob
tf.__version__

'2.10.0'

In [2]:
IMAGE_SIZE = 512
def representative_data_gen():
    dataset_list=glob.glob('VOCdevkit/VOC2007/JPEGImages/*')
    dataset_list_index=np.random.choice(range(len(dataset_list)),100)
    for i in range(100):
        image = dataset_list[dataset_list_index[i]]
        print(image)
        image = tf.io.read_file(image)
        image = tf.io.decode_jpeg(image,channels=3)
        image = tf.image.resize(image,[IMAGE_SIZE, IMAGE_SIZE])
        image = tf.cast(image,tf.float32)
        image = tf.expand_dims(image,0)
        #with tf.Session() as sess:
        #   image = sess.run(image)
        yield [image]

In [3]:
save_keras_model = tf.keras.models.load_model('./export/deeplab.h5',custom_objects={'tf':tf})
save_keras_model.summary()

C:\Users\es402b\anaconda3\envs\EfficientDet\lib\site-packages\keras\layers\core\lambda_layer.py:330: UserWarning: nets.deeplab is not loaded, but a Lambda layer uses it. It may cause errors.
  config, custom_objects, "function", "module", "function_type"


Model: "deeplabv3plus"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv (Conv2D)                  (None, 256, 256, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 Conv_BN (BatchNormalization)   (None, 256, 256, 32  128         ['Conv[0][0]']                   
                                )                                                     

In [4]:
if tf.__version__[0]=='2':
    converter = tf.lite.TFLiteConverter.from_keras_model(save_keras_model)
elif tf.__version__[0]=='1':
    converter = tf.lite.TFLiteConverter.from_keras_model_file('./export/deeplab.h5')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf. lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8 
converter.inference_output_type = tf.uint8
converter.representative_dataset = representative_data_gen 
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\es402b\AppData\Local\Temp\tmp7z1ogj8x\assets


INFO:tensorflow:Assets written to: C:\Users\es402b\AppData\Local\Temp\tmp7z1ogj8x\assets
C:\Users\es402b\anaconda3\envs\EfficientDet\lib\site-packages\tensorflow\lite\python\convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


VOCdevkit/VOC2007/JPEGImages\2011_001687.jpg
VOCdevkit/VOC2007/JPEGImages\2012_001425.jpg
VOCdevkit/VOC2007/JPEGImages\2009_004956.jpg
VOCdevkit/VOC2007/JPEGImages\2012_000891.jpg
VOCdevkit/VOC2007/JPEGImages\2011_002690.jpg
VOCdevkit/VOC2007/JPEGImages\2011_001841.jpg
VOCdevkit/VOC2007/JPEGImages\2011_000901.jpg
VOCdevkit/VOC2007/JPEGImages\2010_005752.jpg
VOCdevkit/VOC2007/JPEGImages\2010_002400.jpg
VOCdevkit/VOC2007/JPEGImages\2010_004451.jpg
VOCdevkit/VOC2007/JPEGImages\2010_001864.jpg
VOCdevkit/VOC2007/JPEGImages\2011_001705.jpg
VOCdevkit/VOC2007/JPEGImages\2012_001198.jpg
VOCdevkit/VOC2007/JPEGImages\2011_001421.jpg
VOCdevkit/VOC2007/JPEGImages\2011_001668.jpg
VOCdevkit/VOC2007/JPEGImages\2008_001349.jpg
VOCdevkit/VOC2007/JPEGImages\2008_007928.jpg
VOCdevkit/VOC2007/JPEGImages\2008_000090.jpg
VOCdevkit/VOC2007/JPEGImages\2012_000632.jpg
VOCdevkit/VOC2007/JPEGImages\2008_005080.jpg
VOCdevkit/VOC2007/JPEGImages\2011_005305.jpg
VOCdevkit/VOC2007/JPEGImages\2010_004657.jpg
VOCdevkit/

In [5]:
with open('deeplab.tflite','wb') as f:
    f.write(tflite_model)